In [311]:
from pyhanlp import *
from pyltp import SentenceSplitter, Segmentor, Postagger, Parser, NamedEntityRecognizer, SementicRoleLabeller
import os

MODELDIR=os.path.join("/Users/xuyong/Downloads/ltp_model")

#import os
import synonyms
from gensim.models import KeyedVectors

# model = KeyedVectors.load_word2vec_format("data/words.vector")
model = KeyedVectors.load_word2vec_format("data/500000-small.txt")
import re

In [288]:
from bert_serving.client import BertClient
from sklearn.metrics.pairwise import cosine_similarity
bert_client = BertClient()

In [307]:
from bert4keras.models import build_transformer_model
from bert4keras.tokenizers import Tokenizer, load_vocab
import numpy as np

In [316]:
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'  # 屏蔽GPU

# config_path: albert模型结构描述文件路径
config_path = '/Users/xuyong/Downloads/albert_model/albert_xxlarge/albert_config.json'
# checkpoint_path: TensorFlow checkpoint文件路径
checkpoint_path = '/Users/xuyong/Downloads/albert_model/albert_xxlarge/model.ckpt-best'
# dict_path: albert词汇表路径
dict_path = '/Users/xuyong/Downloads/albert_model/albert_xxlarge/vocab_chinese.txt'

# 载入预训练ALBERT模型
albert_model = build_transformer_model(config_path, checkpoint_path, model='albert',
                                with_pool=True)

In [114]:
class SentDepPasrse(object):
    def __init__(self):
        self.sen_dep = [[]]
        self.sentence = [[]]
        self.postags = [[]]
      
    def init_parse_sent(self, sent):
        for i in HanLP.parseDependency(sent):
            item = i.toString().split('\t')
            self.sen_dep.append(item)
            self.sentence.append(item[1])
            self.postags.append(item[3])
        

    def get_jiebin(self):
#         print(self.sen_dep)
        jiebin_part = []
        jiebin_id = []
        jiebin_jg = []
        jiebin_zxc_id = []
        for i in self.sen_dep[1:]:
            if i[7] == '介宾关系':
                is_exist = False
                jb_left = int(i[6])
                jb_right = int(i[0])
                for j in jiebin_id:
                    if jb_left == j[0] or jb_left < j[0] and jb_right > j[1]:
                        # 已存在 或者 嵌套
                        is_exist = True
                        break

                if not is_exist:
                    if int(i[0]) < len(self.sen_dep)-1:
                        next_sen = self.sen_dep[int(i[0])+1]
                        # 对xxx而言， 主要是为了寻找 "而言"
                        if next_sen[7] == '右附加关系' and int(next_sen[6]) == jb_left:
                            jb_right = int(next_sen[0])

                    # 判断 介宾短语不能是动词结尾，而且只有两个词
                    if jb_right - jb_left == 1 and self.sen_dep[jb_right][4] == 'v':
                        continue
                    jiebin_jg.append(self.sen_dep[jb_left][7])
                    jiebin_id.append([jb_left, jb_right])
#                     print(jb_left)
                    zxc_id = int(self.sen_dep[jb_left][6])
                    jiebin_zxc_id.append(zxc_id)
#                     print(sen_dep[jb_left])
                    jiebin_part.append(self.sentence[jb_left: jb_right+1])
#         print(jiebin_jg)
        return jiebin_part, jiebin_id, jiebin_jg, jiebin_zxc_id

    
    def find_zhuyu(self, zxc_dict, zxc_id, jb_id, jb_part):
        '''
            介宾短语作状中关系，找主语或者合适的为止
        '''
        # 先找到主语
        zhuyu_id = 0
        zhuyu_part = ''
        for i in self.sen_dep[1:]:
            if int(i[6]) == zxc_id:
                # 找到了同样的中心词 id
                for j in self.sen_dep[int(i[0])-1: 0: -1]:
#                     if j[1] in ',，；;':
#                         # 跳出，只在这个分句前后找主语
#                         break
                    # 往前遍历找主语
                    # 1、直接找到了主谓，并且是名词、代词
                    if j[7] == '主谓关系' and j[4][0] in 'rn' or j[7] == '主谓关系' and j[4] == 'vn' or j[7] == '定中关系' and j[3][0] in 'n' and zxc_id - int(j[0]) == 1: 
                        # 当红军攻打总统府时，随着一声炮响，庞大的建筑在瞬间坍塌，就连蒋介石也对此感到十分震惊。
                        # 在各级党和政府的坚强领导下，社会各界迸发出众志成城抗击疫情的伟力
                        zhuyu_id = j[0]
                        zhuyu_part = j[1]
                        # 找一个就行
                        break
                    elif j[7] == '主谓关系' and j[4] == 'v':
                        # 也许主语在后面
                        for k in self.sen_dep[int(i[0])-1: int(j[0]) : -1]:
                            if int(k[6]) > int(j[0]) and int(k[6]) < zxc_id and k[4][0] in 'nr':
                                # 找到了最靠近中心词的名词，那么就可以了。
                                zhuyu_id = k[0]
                                zhuyu_part = k[1]
                                break
                        if zhuyu_id != 0:
                            break
                    else:
                        continue
                print(zhuyu_id, zhuyu_part)
            if zhuyu_id != 0:
                break
                        
        if zhuyu_id == 0:
            # 没找到主语
            zxc_dict[str(jb_id[0])] = jb_part

        if int(zhuyu_id) < jb_id[0]:
            # 介宾在主语后面，所以要移到主语前面
            # 找到主语分句的逗号
            right_id = 0
            for j in self.sen_dep[int(zhuyu_id): 0: -1]:
                if j[1] in ',，。.；;?？':
                    right_id = j[0]
                    zhuyu_part = j[1]
                    break
                elif j[0] == '1':
                    right_id = j[0]
                    zhuyu_part = '，'
                    break
            if int(right_id) > 0:
                if right_id not in zxc_dict:
                    zxc_dict[right_id] = [zhuyu_part] + [j[1]]
                zxc_dict[right_id] =  jb_part + zxc_dict[right_id] 
        elif int(zhuyu_id) > jb_id[1]:
            # 介宾在主语前面，所以要移到主语后面
            if zhuyu_id not in zxc_dict:
                zxc_dict[zhuyu_id] = [zhuyu_part]
            zxc_dict[zhuyu_id] += jb_part 
        return zxc_dict

    def find_dongci(self, zxc_dict, zxc_id, jb_id, jb_part):
        '''
            介宾短语作动补关系，找动词
        '''
#         print(zxc_dict)

        if jb_id[1] < zxc_id:
            # 放中心词后面
            zxc_dict[str(zxc_id)] += jb_part
        elif jb_id[0] > zxc_id:
            # 放中心词前面
            zxc_dict[str(zxc_id)] = jb_part + zxc_dict[str(zxc_id)]
        return zxc_dict

    def make_zxc_dict(self, jiebin_part, jiebin_id, jiebin_jg, jiebin_zxc_id):
        zxc_dict = {}
        for idx,i in enumerate(jiebin_zxc_id):
            if jiebin_jg[idx] == '动补结构':
                if str(i) not in zxc_dict:
                    zxc_dict[str(i)] = [self.sentence[i]]
                zxc_dict = self.find_dongci(zxc_dict, i, jiebin_id[idx], jiebin_part[idx])
            elif jiebin_jg[idx] == '状中结构': 
                zxc_dict = self.find_zhuyu(zxc_dict, i, jiebin_id[idx], jiebin_part[idx])
            else:
                zxc_dict[str(jiebin_id[idx][0])] = jiebin_part[idx]
        return zxc_dict



    def daozhuang(self, jiebin_id, zxc_dict):
        dz_sentence = []

        # 遍历介宾
        for idx, i in enumerate(self.sen_dep[1:]):
        #     print(i)
            idx += 1
            is_match = False
            for jid, j in enumerate(jiebin_id):
                if idx >= j[0] and idx <= j[1]:
                    # 命中介宾短语
                    is_match = True
                    break
#             print(dz_sentence)
            if str(idx) in zxc_dict:
                dz_sentence += zxc_dict[str(idx)]
                continue
            if not is_match:
                dz_sentence += i[1]
        if dz_sentence[0] in '，,;；。':
            dz_sentence.pop(0)
        return ''.join(dz_sentence)

In [217]:
sent = '从煤层气井生产过程中气、水产量的变化特征可以将煤层气开采阶段划分为三个阶段。'
sent = '副车架纵梁、横梁之间的连接，直接关系到副车架的工作稳定性。'
# sent = '抓好教育工作是关系到国计民生的一件大事。'
# sent = '对于关系到国计民生的大量性建筑，应用技术越简单、越实用，就越便于推广。它的优劣关系到道路功能的发挥和在路网中是否能充分发挥它的作用。从而避免了人为升降国旗与国歌演奏时间不同步的情况发生，确证了国旗升、降仪式的顺利进行。其从成立到至今不到六年的时间里在瞬间这个基础之上，对我造成了伤害，作为一个少年，关系到国家命运。我们要加以保护，往前看，过往的经历，往往很重要，把所有的力量运往前线，深夜里,一个暗影出现在窗前。=窗前出现了一个暗影。事物之间的内在联系，事在人为，人生在世，我们活在这世界上，信道将转化为多个并行的信道，在每个信道上独立传输数据。'
sent = '夹紧力是设计手部的重要参数，直接关系到了我们选择此处所需使用的丝杠步进电机，手部的本体重力和处于工作时所产生的动载荷以及冲击力。'
sent = '由于实际的需要，对它的精度和功能要求也越来越高，这样不仅可以规范升旗过程，使升旗更加方便更加实用，而且可以通过不同的设置满足不同的需求。'
sent = '每一次对作品的改进都是我学习的收获。'
sent = '当红军攻打总统府时，随着一声炮响，庞大的建筑在瞬间坍塌，就连蒋介石也对此感到十分震惊。'
# 因此,为了使倒立摆系统很好的稳定,我们需要施以合适的控制。
print(HanLP.parseDependency(sent))
sen_dep_parser = SentDepPasrse()
sen_dep_parser.init_parse_sent(sent)
jiebin_part, jiebin_id, jiebin_jg, jiebin_zxc_id = sen_dep_parser.get_jiebin()
print( jiebin_part)
print(jiebin_id)
print(jiebin_jg)
print(jiebin_zxc_id)

1	当	当	p	p	_	17	状中结构	_	_
2	红军	红军	n	n	_	3	主谓关系	_	_
3	攻打	攻打	v	v	_	5	定中关系	_	_
4	总统府	总统府	n	n	_	3	动宾关系	_	_
5	时	时	Ng	Ng	_	1	介宾关系	_	_
6	，	，	wp	w	_	1	标点符号	_	_
7	随着	随着	p	p	_	17	状中结构	_	_
8	一	一	m	m	_	9	定中关系	_	_
9	声	声	q	q	_	10	定中关系	_	_
10	炮响	炮响	n	n	_	7	介宾关系	_	_
11	，	，	wp	w	_	7	标点符号	_	_
12	庞大	庞大	a	a	_	14	定中关系	_	_
13	的	的	u	u	_	12	右附加关系	_	_
14	建筑	建筑	v	vn	_	17	主谓关系	_	_
15	在	在	p	p	_	17	状中结构	_	_
16	瞬间	瞬间	nt	t	_	15	介宾关系	_	_
17	坍塌	坍塌	v	v	_	24	主谓关系	_	_
18	，	，	wp	w	_	17	标点符号	_	_
19	就	就	d	d	_	24	状中结构	_	_
20	连	连	u	u	_	24	状中结构	_	_
21	蒋介石	蒋介石	nh	nr	_	24	主谓关系	_	_
22	也	也	d	d	_	24	状中结构	_	_
23	对此	对此	d	d	_	24	状中结构	_	_
24	感到	感到	v	v	_	0	核心关系	_	_
25	十分	十分	d	d	_	26	状中结构	_	_
26	震惊	震惊	v	v	_	24	动宾关系	_	_
27	。	。	wp	w	_	24	标点符号	_	_

[['当', '红军', '攻打', '总统府', '时'], ['随着', '一', '声', '炮响'], ['在', '瞬间']]
[[1, 5], [7, 10], [15, 16]]
['状中结构', '状中结构', '状中结构']
[17, 17, 17]


In [370]:
sent = '中国茶经过漫长历史形成独特茶文化。'

sen_dep = [[]]
sentence = [[]]
postags = [[]]
for i in HanLP.parseDependency(sent):
    item = i.toString().split('\t')
    sen_dep.append(item)
    sentence.append(item[1])
    postags.append(item[3])
sen_dep[1:]




[['1', '中国', '中国', 'ns', 'ns', '_', '2', '定中关系', '_', '_'],
 ['2', '茶', '茶', 'Ng', 'Ng', '_', '6', '主谓关系', '_', '_'],
 ['3', '经过', '经过', 'p', 'p', '_', '6', '状中结构', '_', '_'],
 ['4', '漫长', '漫长', 'a', 'a', '_', '5', '定中关系', '_', '_'],
 ['5', '历史', '历史', 'n', 'n', '_', '6', '主谓关系', '_', '_'],
 ['6', '形成', '形成', 'v', 'v', '_', '0', '核心关系', '_', '_'],
 ['7', '独特', '独特', 'a', 'a', '_', '8', '定中关系', '_', '_'],
 ['8', '茶文化', '茶文化', 'n', 'n', '_', '6', '动宾关系', '_', '_'],
 ['9', '。', '。', 'wp', 'w', '_', '6', '标点符号', '_', '_']]

In [369]:
new_sentence = ''

main_sentence = 'Nnvradpbiw'  # 名词、动词、代词、形容词、副词、介词、连词、区别词、成语、标点

for sen in sen_dep[1:]:
    if sen[3][0] in main_sentence:
        new_sentence += sen[1]
new_sentence

'中国茶经过漫长历史形成独特茶文化。'

In [453]:
def find_best_albert_ww(w1, w2, w3):
    origin_w1w2w3 = w1 + w2 + w3
    w1_syn = get_syn_words(w1)
    w2_syn = get_syn_words(w2)
    w3_syn = get_syn_words(w3)
    
    best_score = -1
    best_w1w2w3 = ''
    
    tokenizer = Tokenizer(dict_path, do_lower_case=True)
    token_ids, segment_ids = tokenizer.encode(origin_w1w2w3)
    v1 = albert_model.predict([np.array([token_ids]), np.array([segment_ids])])
    for w_1 in w1_syn:
        for w_2 in w2_syn:
            for w_3 in w3_syn:
                cur_w1w2w3 = w_1 + w_2 + w_3
                token_ids2, segment_ids2 = tokenizer.encode(cur_w1w2w3)
                v2 = albert_model.predict([np.array([token_ids2]), np.array([segment_ids2])])
                tensors = np.vstack((v1,v2))
                score = cosine_similarity(tensors)[0][1]
    #             print(cur_w1w2, score)
                if score > best_score:
                    best_w1w2 = cur_w1w2w3
                    best_score = score
    return best_w1w2, best_score

# 5.14

In [452]:
# 相邻的定中
def dingzhong_rel(sent):
    
    dep = [[]]

    for i in HanLP.parseDependency(sent):
        item = i.toString().split('\t')
        dep.append(item)
        
    new_sentence = ''

    main_sentence = 'Nnvradpbimqw'  # 名词、动词、代词、形容词、副词、介词、连词、区别词、成语、数词、量词、标点

    for sen in dep[1:]:
        if sen[3][0] in main_sentence:
            new_sentence += sen[1]
 
    sen_dep = [[]]
    for i in HanLP.parseDependency(new_sentence):
        item = i.toString().split('\t')
        sen_dep.append(item)

    dingzhong = []
    idx = -1
    for i in sen_dep[1:]:
        if i[0] != idx and i[3] not in 'mq' and i[7] == '定中关系' and int(i[6]) == int(i[0]) + 1:
            dingzhong.append([i[1], sen_dep[int(i[6])][1]])
            idx = i[6]
        elif i[0] != idx:
            dingzhong.append([i[1]])
    return dingzhong

sent = '从煤层气井生产过程中气、水产量的变化特征可以将煤层气开采阶段划分为三个阶段。'
dingzhong_rel(sent)



[['从'],
 ['煤层', '气井'],
 ['生产', '过程'],
 ['中气'],
 ['、'],
 ['水产量'],
 ['变化', '特征'],
 ['可以'],
 ['将'],
 ['煤层气'],
 ['开采', '阶段'],
 ['划分'],
 ['为'],
 ['三'],
 ['个'],
 ['阶段'],
 ['。']]

In [449]:
# print(HanLP.parseDependency(sent))

In [454]:
%%time
def find_best_ww(w1, w2, sep = ' '):
    origin_w1w2 = w1 + sep + w2
    w1_syn = get_syn_words(w1)
    w2_syn = get_syn_words(w2)
    
    best_score = -1
    best_w1w2 = origin_w1w2
    
    for w_1 in w1_syn:
        for w_2 in w2_syn:
            cur_w1w2 = w_1 + sep + w_2
            query_list = [origin_w1w2, cur_w1w2]
            tensors = bert_client.encode(query_list)
            score = cosine_similarity(tensors)[0][1]
#             print(cur_w1w2, score)
            if score > best_score:
                best_w1w2 = cur_w1w2
                best_score = score
    return best_w1w2, best_score

# 进行替换
def jiangchong(sent):
    dingzhong_sent = dingzhong_rel(sent)
    new_sent = ''
    for i in dingzhong_sent:
        if len(i) == 2:
            # 定中，可以加 '的'
            new_sent += find_best_ww(i[0], i[1], '的')[0]
        else:
            new_sent += i[0]
    return new_sent

# sent = '从煤层气井生产过程中气、水产量的变化特征可以将煤层气开采阶段划分为三个阶段。'
# jiangchong(sent)

CPU times: user 9 µs, sys: 13 µs, total: 22 µs
Wall time: 43.9 µs


# 测试ltp-service

In [219]:
import requests
import time

In [225]:
test_sentences = "经济管理现代化的发展趋势还体现在团队协作精神的培养上，强烈的团队协作精神是现代化经济管理体系的重要特色。" \
                     "当红军攻打总统府时，随着一声炮响，庞大的建筑在瞬间坍塌，就连蒋介石也对此感到十分震惊。" \
                     "中国红军过草地、爬雪山，历经了千辛万苦才摆脱国民党的围追堵截，来到了延安，这片神奇又包容的土地。" \
                     "毛泽东带领中国红军去了延安，找到了革命的方向，为中国革命征程掀开了崭新的一页！" \
                     "毛泽东有许多著作都是在延安完成的，例如《新民主主义论》、《实践论》、《矛盾论》以及《论持久战》等。" \
                     "同时，毛泽东思想也是在延安时期逐步走向成熟并写入党的旗帜上的。" \
                     "经过漫漫长征的中国共产党在延安扎根之时，困难重重。" \
                     "支撑中国共产党领导延安人民在物资极其匮乏、生产力极其落后的条件下，大力发展生产，开垦荒田，化腐朽为神奇的，正是信仰，是一种为实现共产主义不怕流血、不怕牺牲的信念。" \
                     "物质日益丰富的我们缺少的正是思想、物质和素质。" \
                     "毛泽东的名言：“站在最大多数劳动人民一边。" \
                     "就如毛泽东说的那样：一切反动派都是纸老虎。" \
                     "从仿真中我们可以看到,在未加控制的情况下,倒立摆系统是极不稳定的。" \
                     "在中国，是历经二万五千里长征的红军掌握了人民，掌握了革命胜利的方向。" \
                     "中国共产党之所以能够取得战争的最终胜利，就是因为紧紧的依靠人民群众的主体地位，最大程度的保障人民群众的利益，同时秉持着实事求是、艰苦奋斗的革命作风。" \
                     "全心全意为人民服务的根本宗旨；自力更生,艰苦奋斗的创业精神；解放思想,实事求是的思想路线。" \
                     "全心全意为人民服务是延安精神的本质，是党的政治立场和行为准则，也是党的根本宗旨。" \
                     "在延安的13年里，小米饭滋养着我们八路的战士们，他们在这里得以休养生息，得以自力更生，开辟了陕甘宁根据地，慢慢创造出了中国的新生力量。在延安，我们看到了中国无数精神的体现，这些精神是我们后辈永远值得学习的。当年在延安，革命先烈们为了人民的新生活，不惜洒热血，抛头颅，将延安时代发展成了延安精神。那延安精神到底是什么呢？延安精神是共产党人站高看远的崇高信仰；延安精神是对抗谬论的斗争精神；延安精神是吃苦耐劳的创造精神；延安精神是团结就是力量的集体主义精神；延安精神是全心全意为人民的服务精神。" \
                     "新型冠状病毒是2019年末发现的最为严重的病毒。随着新年的到来，一场没有硝烟的战争也开始打响。在各级党和政府的坚强领导下，社会各界迸发出众志成城抗击疫情的伟力。一方有难，八方支援，这就是团结的力量，也是在延安时代，我们学习总结出来的。"\
                     "所以对于自助快递柜可以适当地减少部分过高成本或者不必要的技术，可以针对用户需求，相对增加或减少区域自助快递柜的数量，提高利用率。做到简单、方便、适用就可以。可以大大减少配送员的压力。对于维修成本，公司可以组织负责区域维修专员，可按月度、季度、年度对所管辖区域的快递柜进行定期检修、批量维修。不仅能够确保自助快递柜达到长期有效运作，也能够减少故障等现象的出现，从而减少维护成本。"\
                    '正是由于西方社会在发展经济的过程中对环境造成了破坏，才使得人们开始反思人与自然的相处关系，进而发展到对动物保护的高度关注。'\
                    '这件事情对一个战士的思想感情造成了伤害。'\
                    '从煤层气井生产过程中气、水产量的变化特征可以将煤层气开采阶段划分为三个阶段。'\
                    '对信道矩阵进行特征值分解，信道将转化为多个并行的信道，在每个信道上独立传输数据。'\
                    '1657年,他写成了一本关于自己研究问题的专著《论赌博中的计算》'\
                    '由于实际的需要，对它的精度和功能要求也越来越高，这样不仅可以规范升旗过程，使升旗更加方便更加实用，而且可以通过不同的设置满足不同的需求。'\
                    '每一次对作品的改进都是我学习的收获。'\
                    "在这种教养方式下成长的孩子不善于交朋友，也不善于表达，做任何事情都小心翼翼，生怕做得不好会受别人的批评。"\
                    "冠状病毒病在武汉暴发流行后，方舱医院为控制疫情扩散发挥了重要作用。"\
                    "电气主接线是变电所电气设计的首要任务，也是构成电力系统的重要环节。"\
                    "全面分析有关影响因素，通过技术经济比较，确定变电所主接线的最佳方案。"\
                    "负荷大小和重要性：对于一级负荷必须有两个独立电源供电，且当任何一个电源失去后，能保证全部一级负荷不间断供电，对于二级负荷一般也要两个独立电源供电，且当任何一个电源失去后，能保证全部或大部分二级负荷的供电，对于三级负荷一般只需一个独立电源供电。"\
                    "货币资金是医院资产的重要组成部分,贯穿于医院管理的全过程。"\
                    "现今社会中，框架结构广泛应用于办公和公用高层建筑，也大量应用于高层旅馆设计，我的毕业设计是框架结构的高层办公楼设计。 "\
                    "虽然独立结构的荷载是经常被注意到的问题,但是在进行框架的建设中,却经常出现漏载,在进行构建时也较容易产生纰漏,从而导致结构设计建设上出现了不合理。"\
                    "针对上述问题,本文从完善县级经济责任审计制度,完善其审计工作,突出审计重点和特点,改进审计方法,提高审计质量,强化问责意识,树立审计权威等方面提出了对策和建议。"\
                    "在很多人看来，今年注定是不平凡的一年，这一不平凡之于本人，甚是如此。"\
                    "层次分析法得到的相关矩阵，都需要通过一致性比较，如果没有进行一致性比较，没有在一致性指标的数据范围内，层次分析法将无法得到预期的结果；"\
                    "多年来,越来越受到地方各级党委、政府和社会各界的重视。"\
                    "针对新的政治经济形势,改进政府作风,推进廉政建设,使我们更加有效地开展审计工作的重要任务。"\
                    "该理论认为，资本结构与企业的价值无关，决定企业价值高低的关键要素是企业的净营业收益。"\
                    "以内部融资为主要融资方式的企业可以有效控制财务风险，保持稳健的财务状况。"


In [ ]:
btime = time.time()

sents = re.split('([。！])', test_sentences)

for i in range(0, len(sents)-1, 2):
    params['sentence'] = sents[i]+sents[i+1]
    stime = time.time()
    r = requests.get(url, params=params)
    print("\n\n****原句****: ", sents[i]+sents[i+1] )
    print('****改句****: ', r.text)
    print('****去重时间**** %.2fs' % (time.time() - stime))
print('====总耗时==== %.2fs ' % (time.time() - stime))

In [458]:
btime = time.time()

host = "http://127.0.0.1:4259/"
endpoint = "jiangchong"
url = ''.join([host, endpoint])
params = {}

sents = re.split('([。！])', test_sentences)

for i in range(0, len(sents)-1, 2):
    sent = sents[i]+sents[i+1]
    print("\n\n****原句****: ", sent)
    r = jiangchong(sent)
    print('****改句****: ', r)
    print('****去重时间**** %.2fs' % (time.time() - stime))
print('====总耗时==== %.2fs ' % (time.time() - stime))



****原句****:  经济管理现代化的发展趋势还体现在团队协作精神的培养上，强烈的团队协作精神是现代化经济管理体系的重要特色。
****改句****:  经济经营管理的最现代化发展趋势还体现在团队协作精神培养上，更强烈的的团队协作精神是最现代化的整体经济管理体系极为重要的独有特色。
****去重时间**** 910514.72s


****原句****:  当红军攻打总统府时，随着一声炮响，庞大的建筑在瞬间坍塌，就连蒋介石也对此感到十分震惊。
****改句****:  当红军攻打总统府时，随着一声炮响，数量庞大的整个建筑在瞬间坍塌，就蒋介石也对此感到十分震惊。
****去重时间**** 910518.86s


****原句****:  中国红军过草地、爬雪山，历经了千辛万苦才摆脱国民党的围追堵截，来到了延安，这片神奇又包容的土地。
****改句****:  说中国的中央红军过草地、爬雪山，历经千辛万苦才摆脱国民党的围追堵截，来到延安，这的片神奇包容土地。
****去重时间**** 910521.04s


****原句****:  毛泽东带领中国红军去了延安，找到了革命的方向，为中国革命征程掀开了崭新的一页！
****改句****:  毛泽东带领说中国的中央红军去延安，找到大革命的前进方向，为中国大革命的新征程掀开崭新一页！
****去重时间**** 910531.15s


****原句****:  毛泽东有许多著作都是在延安完成的，例如《新民主主义论》、《实践论》、《矛盾论》以及《论持久战》等。
****改句****:  毛泽东有许多著作都是在延安完成，例如《新民主主义论》、《实践论》、《内部矛盾的若论》《论持久战》。
****去重时间**** 910533.83s


****原句****:  同时，毛泽东思想也是在延安时期逐步走向成熟并写入党的旗帜上的。
****改句****:  ，毛泽东思想也是在在延安的该时期逐步走向成熟写入劳动党的一面旗帜上。
****去重时间**** 910536.86s


****原句****:  同时，毛泽东思想也是在延安的时候才逐步走向成熟并写入党的旗帜上的。
****改句****:  ，毛泽东思想也是在延安的时候才逐步走向成熟写入劳动党的一面旗帜上。
****去重时间**** 910537.82s


****原